In [2]:
import torch
import pandas as pd
from torch.utils.data import Dataset

In [3]:
data_df = pd.read_csv('combined_lfud_huggingface.csv')

In [4]:
data_df.head()

,sentence,fallacy_type,sentence_ro,proposition,proposition_ro
0,"company's slogan ""Expect More. Pay Less.""",appeal to emotion,sloganul companiei „Așteptați mai mult. Plătiț...,NaN,NaN
1,"The bigger a child's shoe size, the better the...",false causality,Cu cât mărimea pantofilor unui copil este mai ...,NaN,NaN
2,"Since many people believe this, then it must b...",ad populum,"Din moment ce mulți oameni cred asta, atunci t...",NaN,NaN
3,Senator Randall isn't lying when she says she ...,circular reasoning,Senatorul Randall nu minte când spune că îi pa...,NaN,NaN
4,A mother is telling her daughter that she went...,fallacy of relevance,O mamă îi spune fiicei ei că și-a analizat dat...,NaN,NaN


In [5]:
dataset = data_df[['fallacy_type', 'sentence_ro']]

In [6]:
dataset.head()

,fallacy_type,sentence_ro
0,appeal to emotion,sloganul companiei „Așteptați mai mult. Plătiț...
1,false causality,Cu cât mărimea pantofilor unui copil este mai ...
2,ad populum,"Din moment ce mulți oameni cred asta, atunci t..."
3,circular reasoning,Senatorul Randall nu minte când spune că îi pa...
4,fallacy of relevance,O mamă îi spune fiicei ei că și-a analizat dat...


In [7]:
dataset['fallacy_type'].value_counts()

,count
fallacy_type,
faulty generalization,641
intentional,547
ad hominem,465
appeal to emotion,375
ad populum,362
false causality,356
fallacy of relevance,332
fallacy of credibility,330
fallacy of extension,272


In [8]:
size = dataset.shape[0]
print(size)
size_train = int(0.7 * size)
size_test = int(0.15 * size)

train_dataset = dataset.loc[:size_train-1, :]
test_dataset = dataset.loc[size_train:size_train+size_test-1, :]
dev_dataset = dataset.loc[size_train+size_test: :]

train_dataset.shape, test_dataset.shape, dev_dataset.shape

4565


((3195, 2), (684, 2), (686, 2))

In [9]:
logical_fallacies = ['faulty generalization', 'intentional', 'ad hominem']
train_data = train_dataset[train_dataset.fallacy_type.isin(logical_fallacies)]
test_data = test_dataset[test_dataset.fallacy_type.isin(logical_fallacies)]
dev_data = dev_dataset[dev_dataset.fallacy_type.isin(logical_fallacies)]

In [10]:
len(train_data), len(test_data), len(dev_data)

(1218, 263, 172)

In [11]:
num_labels = len(logical_fallacies)

id2label = {id: label for id, label in enumerate(logical_fallacies)}

label2id = {label: id for id, label in enumerate(logical_fallacies)}

In [12]:
label2id

{'faulty generalization': 0, 'intentional': 1, 'ad hominem': 2}

In [13]:
id2label

{0: 'faulty generalization', 1: 'intentional', 2: 'ad hominem'}

In [14]:
train_data.loc[:, "fallacy_type_id"] = train_data['fallacy_type'].apply(lambda x: label2id[x.strip()])
test_data.loc[:, "fallacy_type_id"] = test_data['fallacy_type'].apply(lambda x: label2id[x.strip()])
dev_data.loc[:, "fallacy_type_id"] = dev_data['fallacy_type'].apply(lambda x: label2id[x.strip()])

<ipython-input-14-04c59070ce85>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.loc[:, "fallacy_type_id"] = train_data['fallacy_type'].apply(lambda x: label2id[x.strip()])
<ipython-input-14-04c59070ce85>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, "fallacy_type_id"] = test_data['fallacy_type'].apply(lambda x: label2id[x.strip()])
<ipython-input-14-04c59070ce85>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [15]:
test_data.head()

,fallacy_type,sentence_ro,fallacy_type_id
3195,ad hominem,Aceasta este o eroare a irelevanței care se ba...,2
3196,ad hominem,Helga: Nu ar trebui să mănânci așa... s-a dove...,2
3197,ad hominem,Bill: „Cred că avortul este greșit din punct d...,2
3198,faulty generalization,Dacă mai folosim o cutie de fixativ luna aceas...,0
3201,ad hominem,„De ce ați alege un primar a cărui soție a fos...,2


In [16]:
from transformers import BertTokenizerFast, BertTokenizer

# ro-bert
tokenizer = BertTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-uncased-v1", max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/411k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
from transformers import BertForSequenceClassification
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'
model = BertForSequenceClassification.from_pretrained("dumitrescustefan/bert-base-romanian-uncased-v1",
                                                      num_labels=num_labels,
                                                      id2label=id2label, label2id=label2id)

# Make model weights contiguous
for name, param in model.named_parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()
model.to(device)

pytorch_model.bin:   0%|          | 0.00/500M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dumitrescustefan/bert-base-romanian-uncased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [19]:
train_encodings = tokenizer(list(train_data['sentence_ro']), padding=True, truncation=True,  max_length=512)
test_encodings = tokenizer(list(test_data['sentence_ro']), padding=True, truncation=True,  max_length=512)
dev_encodings = tokenizer(list(dev_data['sentence_ro']), padding=True, truncation=True,  max_length=512)

In [20]:
train_labels = list(train_data['fallacy_type_id'])
test_labels = list(test_data['fallacy_type_id'])
dev_labels = list(dev_data['fallacy_type_id'])

In [21]:
class DataLoader(Dataset):
    """
    Custom Dataset class for handling tokenized text data and corresponding labels.
    Inherits from torch.utils.data.Dataset.
    """

    def __init__(self, encodings, labels):
        """
        Initializes the DataLoader class with encodings and labels.

        Args:
            encodings (dict): A dictionary containing tokenized input text data
                              (e.g., 'input_ids', 'token_type_ids', 'attention_mask').
            labels (list): A list of integer labels for the input text data.
        """
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        """
        Returns a dictionary containing tokenized data and the corresponding label for a given index.

        Args:
            idx (int): The index of the data item to retrieve.

        Returns:
            item (dict): A dictionary containing the tokenized data and the corresponding label.
        """
        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item
        # item = {key: torch.tensor(val[idx]).contiguous() for key, val in self.encodings.items()}
        # item['labels'] = torch.tensor(self.labels[idx]).contiguous()
        # return item

    def __len__(self):
        """
        Returns the number of data items in the dataset.

        Returns:
            (int): The number of data items in the dataset.
        """
        return len(self.labels)

In [22]:
train_dataloader = DataLoader(train_encodings, train_labels)
test_dataloader = DataLoader(test_encodings, test_labels)
dev_dataloader = DataLoader(dev_encodings, dev_labels)

In [23]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    """
    Computes accuracy, F1, precision, and recall for a given set of predictions.

    Args:
        pred (obj): An object containing label_ids and predictions attributes.
            - label_ids (array-like): A 1D array of true class labels.
            - predictions (array-like): A 2D array where each row represents
              an observation, and each column represents the probability of
              that observation belonging to a certain class.

    Returns:
        dict: A dictionary containing the following metrics:
            - Accuracy (float): The proportion of correctly classified instances.
            - F1 (float): The macro F1 score, which is the harmonic mean of precision
              and recall. Macro averaging calculates the metric independently for
              each class and then takes the average.
            - Precision (float): The macro precision, which is the number of true
              positives divided by the sum of true positives and false positives.
            - Recall (float): The macro recall, which is the number of true positives
              divided by the sum of true positives and false negatives.
    """
    # Extract true labels from the input object
    labels = pred.label_ids

    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)

    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')

    # Calculate the accuracy score using sklearn's accuracy_score function
    acc = accuracy_score(labels, preds)

    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [24]:
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    # The pre-trained model that will be fine-tuned
    model=model,
    # Training arguments that we defined above
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=dev_dataloader,
    compute_metrics=compute_metrics
)

In [25]:
# Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.865281,0.627907,0.519476,0.748832,0.631377
2,No log,0.642909,0.761628,0.740130,0.818551,0.763662
3,No log,0.582016,0.784884,0.769791,0.822694,0.786651


TrainOutput(global_step=231, training_loss=0.6459744936460025, metrics={'train_runtime': 290.9251, 'train_samples_per_second': 12.56, 'train_steps_per_second': 0.794, 'total_flos': 651584962201572.0, 'train_loss': 0.6459744936460025, 'epoch': 3.0})

In [26]:
q = [trainer.evaluate(eval_dataset=dataloader) for dataloader in [dev_dataloader, test_dataloader]]
print(q)
results_df = pd.DataFrame(q, index=["dev", "test"])

[{'eval_loss': 0.5820155739784241, 'eval_Accuracy': 0.7848837209302325, 'eval_F1': 0.7697911214398871, 'eval_Precision': 0.822693781314471, 'eval_Recall': 0.7866505343819319, 'eval_runtime': 0.5505, 'eval_samples_per_second': 312.445, 'eval_steps_per_second': 19.982, 'epoch': 3.0}, {'eval_loss': 0.6277555227279663, 'eval_Accuracy': 0.7376425855513308, 'eval_F1': 0.7366789242194823, 'eval_Precision': 0.7472664118058124, 'eval_Recall': 0.7322779769016328, 'eval_runtime': 6.5338, 'eval_samples_per_second': 40.252, 'eval_steps_per_second': 2.602, 'epoch': 3.0}]


In [27]:
print(results_df.iloc[:, :5])

      eval_loss  eval_Accuracy   eval_F1  eval_Precision  eval_Recall
dev    0.582016       0.784884  0.769791        0.822694     0.786651
test   0.627756       0.737643  0.736679        0.747266     0.732278


In [28]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast


def predict(text):
    """
    Predicts the class label for a given input text

    Args:
        text (str): The input text for which the class label needs to be predicted.

    Returns:
        probs (torch.Tensor): Class probabilities for the input text.
        pred_label_idx (torch.Tensor): The index of the predicted class label.
        pred_label (str): The predicted class label.
    """
    # Tokenize the input text and move tensors to the GPU if available
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

    # Get model output (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    """ Explanation outputs: The BERT model returns a tuple containing the output logits (and possibly other elements depending on the model configuration). In this case, the output logits are the first element in the tuple, which is why we access it using outputs[0].

    outputs[0]: This is a tensor containing the raw output logits for each class. The shape of the tensor is (batch_size, num_classes) where batch_size is the number of input samples (in this case, 1, as we are predicting for a single input text) and num_classes is the number of target classes.

    softmax(1): The softmax function is applied along dimension 1 (the class dimension) to convert the raw logits into class probabilities. Softmax normalizes the logits so that they sum to 1, making them interpretable as probabilities. """

    # Get the index of the class with the highest probability
    # argmax() finds the index of the maximum value in the tensor along a specified dimension.
    # By default, if no dimension is specified, it returns the index of the maximum value in the flattened tensor.
    pred_label_idx = probs.argmax()

    # Now map the predicted class index to the actual class label
    # Since pred_label_idx is a tensor containing a single value (the predicted class index),
    # the .item() method is used to extract the value as a scalar
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [29]:
model_path = "logical-fallacies-bert-3-classes-model-full-data"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('logical-fallacies-bert-3-classes-model-full-data/tokenizer_config.json',
 'logical-fallacies-bert-3-classes-model-full-data/special_tokens_map.json',
 'logical-fallacies-bert-3-classes-model-full-data/vocab.txt',
 'logical-fallacies-bert-3-classes-model-full-data/added_tokens.json')

In [30]:
from transformers import pipeline

model_path = "logical-fallacies-bert-3-classes-model-full-data"

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)
nlp = pipeline("text-classification", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [31]:
nlp("Toate florile nu rămân deschise pentru totdeauna. Trandafirii sunt un tip de plante. Prin urmare, toate plantele nu rămân deschise pentru totdeauna..")

# faulty generalization

[{'label': 'faulty generalization', 'score': 0.9158141016960144}]

In [32]:
nlp("Din moment ce nimeni nu-mi poate arăta o floare care rămâne deschisă pentru totdeauna, toate florile nu rămân deschise pentru totdeauna.")

# intentional fallacy

[{'label': 'faulty generalization', 'score': 0.7998501658439636}]

In [33]:
nlp("Toate produsele electronice au nevoie de electricitate.")
# faulty generalization

[{'label': 'faulty generalization', 'score': 0.5407205820083618}]

In [34]:
nlp("Jake susține că toți cei din familia lui nu au fost niciodată în Europa. Dar Jake este un bărbat needucat, așa că afirmația lui trebuie să fie falsă.")
# ad hominem

[{'label': 'ad hominem', 'score': 0.6410394310951233}]

In [35]:
nlp("Nu există rapoarte care să ateste că cineva a avut probleme cu nerespectarea regulilor rutiere; prin urmare, toți oamenii ar trebui să respecte regulile de drum.")

# intentional

[{'label': 'faulty generalization', 'score': 0.8167915940284729}]